### In this project aerial drone images from burned and intact homes from the 2018 California wildfires were used to create a neural network image classifier

## 1. Importing Modules

In [4]:
from keras.models import Sequential

Imported Sequential from keras.models, to initialise our neural network model as a sequential network. There are two basic ways of initialising a neural network, either by a sequence of layers or as a graph.

In [5]:
from keras.layers import Conv2D

Imported Conv2D from keras.layers, this is to perform the convolution operation i.e the first step of a CNN, on the training images. Since we are working on images here, which a basically 2 Dimensional arrays, we’re using Convolution 2-D, you may have to use Convolution 3-D while dealing with videos, where the third dimension will be time.

In [6]:
from keras.layers import MaxPooling2D

Imported MaxPooling2D from keras.layers, which is used for pooling operation, that is the step — 2 in the process of building a cnn. For building this particular neural network, we are using a Maxpooling function, there exist different types of pooling operations like Min Pooling, Mean Pooling, etc. Here in MaxPooling we need the maximum value pixel from the respective region of interest.

In [7]:
from keras.layers import Flatten

Imported Flatten from keras.layers, which is used for Flattening. Flattening is the process of converting all the resultant 2 dimensional arrays into a single long continuous linear vector.

In [8]:
from keras.layers import Dense

Imported Dense from keras.layers, which is used to perform the full connection of the neural network, which is the step 4 in the process of building a CNN.

## 2. Shaping the neural network

Initializing an empty object of sequential type, and then added a convolution layer by using the “Conv2D” function. The Conv2D function is taking 4 arguments, the first is the number of filters i.e 32 here, the second argument is the shape each filter is going to be i.e 3x3 here, the third is the input shape and the type of image(RGB or Black and White)of each image i.e the input image our CNN is going to be taking is of a 64x64 resolution and “3” stands for RGB, which is a colour img, the fourth argument is the activation function we want to use, here ‘relu’ stands for a rectifier function.

In [22]:
classifier = Sequential()

In [23]:
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))

Performing a pooling operation to reduce the size of images as much as possible.

In [24]:
classifier.add(MaxPooling2D(pool_size = (2, 2)))

Converting all the pooled images to a continuous vector using flattening.

In [25]:
classifier.add(Flatten())

In this step we created a fully connected layer, and to this layer we connected the set of nodes we got after the flattening step, these nodes will act as an input layer to these fully-connected layers. As this layer will be present between the input layer and output layer, we can refer to it a hidden layer.

Dense is the function to add a fully connected layer, ‘units’ is where we define the number of nodes that should be present in this hidden layer, these units value will be always between the number of input nodes and the output nodes but the art of choosing the most optimal number of nodes can be achieved only through experimental tries. Though it’s a common practice to use a power of 2. And the activation function will be a rectifier function.

In [26]:
classifier.add(Dense(units = 128, activation = 'relu'))

Now it’s time to initialise our output layer, which should contain only one node, as it is binary classification. This single node will give us a binary output of either a Burnt or Intact.

In [27]:
classifier.add(Dense(units = 1, activation = 'sigmoid'))

##### Compiling the model:  
-Optimizer parameter is to choose the stochastic gradient descent algorithm.  
-Loss parameter is to choose the loss function.  
-Finally, the metrics parameter is to choose the performance metric.

In [28]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

## 3. Fitting the model

Before fitting we pre-process the images to prevent overfitting, that is when the model has great training accuracy, but poor testing performance.

So before we fit our images to the neural network, we need to perform some image augmentations on them, which is basically synthesising the training data. We are going to do this using keras.preprocessing library for doing the synthesising part as well as to prepare the training set as well as the test test set of images that are present in a properly structured directories, where the directory’s name is take as the label of all the images present in it. For example : All the images inside the ‘Burned’ named folder will be considered as burned homes by keras.

In [29]:
from keras.preprocessing.image import ImageDataGenerator

Creating synthetic data out of the same images by performing different type of operations on these images like flipping, rotating, blurring, etc.

In [30]:
train_datagen = ImageDataGenerator(rescale = 1./255,
    shear_range = 0.2,
        zoom_range = 0.2,
            horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('training_set',
    target_size = (64, 64),
        batch_size = 32,
            class_mode = 'binary') # Setting to categorical because we got 3 classes
test_set = test_datagen.flow_from_directory('test_set',
    target_size = (64, 64),
        batch_size = 32,
            class_mode = 'binary')

Found 31 images belonging to 2 classes.
Found 6 images belonging to 2 classes.


#### Fitting the data to the model

In [31]:
classifier.fit_generator(training_set,
steps_per_epoch = 120,
epochs = 5,
validation_data = test_set,
validation_steps = 20)

Epoch 1/5
120/120 [==============================] - 211s 2s/step - loss: 0.1459 - acc: 0.9497 - val_loss: 2.2541 - val_acc: 0.6667
Epoch 2/5
120/120 [==============================] - 198s 2s/step - loss: 0.0032 - acc: 1.0000 - val_loss: 3.4604 - val_acc: 0.6667
Epoch 3/5
120/120 [==============================] - 187s 2s/step - loss: 0.0012 - acc: 1.0000 - val_loss: 3.6536 - val_acc: 0.6667
Epoch 4/5
120/120 [==============================] - 187s 2s/step - loss: 5.9404e-04 - acc: 1.0000 - val_loss: 3.8876 - val_acc: 0.6667
Epoch 5/5
120/120 [==============================] - 187s 2s/step - loss: 3.3330e-04 - acc: 1.0000 - val_loss: 3.9879 - val_acc: 0.6667


In the above code, ‘steps_per_epoch’ holds the number of training images, i.e the number of images the training_set folder contains.
And ‘epochs’, a single epoch is a single step in training a neural network; in other words when a neural network is trained on every training samples only in one pass we say that one epoch is finished. So training process should consist more than one epochs.

#### Making new predictions!

In [35]:
import numpy as np
from keras.preprocessing import image

In [36]:
def predictor(imagefile):
    test_image = image.load_img(imagefile, target_size = (64, 64))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = classifier.predict(test_image)
    training_set.class_indices
    if result[0][0] == 1:
        prediction = 'Burned'
    else:
        prediction = 'Intact'
    return(prediction)

In [38]:
res=[]
act = ['Intact', 'Burned','Burned','Intact']
for i in range(1,5):
    res.append(predictor(f'house{i}.jpg'))
print(res)
print(act)

['Burned', 'Burned', 'Intact', 'Burned']
['Intact', 'Burned', 'Burned', 'Intact']
